In [1]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# Install Flash Attention 2 for GPUs with CUDA capability >= 8 (e.g., A100, H100)
import torch
if torch.cuda.get_device_capability()[0] >= 8:
    !pip install --no-deps packaging ninja einops "flash-attn>=2.6.3"  # Flash Attention 2 for faster training

In [2]:
import torch  # For GPU operations and tensor computations
import os     # For file and directory operations
import re     # For regular expressions
from typing import List, Literal, Optional  # For type hints
from datasets import load_dataset
from unsloth import FastLanguageModel, is_bfloat16_supported  # For efficient model loading and training
from trl import KTOConfig, KTOTrainer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [5]:
# Set basic parameters
max_seq_length = 4096  # Maximum sequence length the model can handle
dtype = None           # Auto-detect data type (float16 for Tesla T4/V100, bfloat16 for Ampere+ GPUs)
load_in_4bit = True    # Use 4-bit quantization to reduce memory usage

# Load the pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,  # Set max sequence length
    dtype=dtype,                    # Auto-detect data type
    load_in_4bit=load_in_4bit,      # Enable 4-bit quantization
    # token="hf_...",                # Use this if accessing gated models (e.g., LLaMA 2)
)

# Add a default chat template if missing
if tokenizer.chat_template is None:
    DEFAULT_CHAT_TEMPLATE = """
    {% for message in messages %}
    {% if message['role'] == 'user' %}
    {{ '<|user|>\n' + message['content'] + eos_token }}
    {% elif message['role'] == 'system' %}
    {{ '<|system|>\n' + message['content'] + eos_token }}
    {% elif message['role'] == 'assistant' %}
    {{ '<|assistant|>\n' + message['content'] + eos_token }}
    {% endif %}
    {% if loop.last and add_generation_prompt %}
    {{ '<|assistant|>' }}
    {% endif %}
    {% endfor %}
    """
    tokenizer.chat_template = DEFAULT_CHAT_TEMPLATE  # Apply the default template++

==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.81G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/236 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/6.77k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [6]:
# Set basic parameters
max_seq_length = 4096  # Maximum sequence length the model can handle
dtype = None           # Auto-detect data type (float16 for Tesla T4/V100, bfloat16 for Ampere+ GPUs)
load_in_4bit = True    # Use 4-bit quantization to reduce memory usage

# Load the pre-trained model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/DeepSeek-R1-Distill-Qwen-1.5B-unsloth-bnb-4bit",
    max_seq_length=max_seq_length,  # Set max sequence length
    dtype=dtype,                    # Auto-detect data type
    load_in_4bit=load_in_4bit,      # Enable 4-bit quantization
    # token="hf_...",                # Use this if accessing gated models (e.g., LLaMA 2)
)


==((====))==  Unsloth 2025.2.4: Fast Qwen2 patching. Transformers: 4.48.3.
   \\   /|    GPU: Tesla P100-PCIE-16GB. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 6.0. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [25]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass


In [27]:
from datasets import load_dataset
ds = load_dataset("lavita/medical-qa-datasets", "all-processed")


In [36]:
print("hello")
train_dataset = ds["train"] 
train_subset = train_dataset.select(range(9000))
train_dataset = train_subset.map(formatting_prompts_func, batched = True,)

hello


Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

In [37]:
train_dataset.shape

(9000, 5)

In [38]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        # max_steps = 60,
        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none", # Use this for WandB etc

    ),
)

Map (num_proc=2):   0%|          | 0/9000 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 9,000 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 1,125
 "-____-"     Number of trainable parameters = 18,464,768


Step,Training Loss
1,2.308100
2,2.587700
3,2.440500
4,2.205800
5,2.457000
6,2.675100
7,2.414200
8,2.061700
9,2.487900
10,2.195700
